In [1]:
manufacturer = 'MELLING'
type_code = '5756'

from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

df = pd.read_excel('./menu.xlsx',
                   header=0,
                   dtype=str).fillna('')

list_url = df['Url'].tolist()
list_part_number = df['Part_Number'].tolist()

print('总数量：' + str(len(list_url)))
print()

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

import requests

from bs4 import BeautifulSoup

from lxml import etree

import numpy as np

import json

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for url, part_number in zip(list_url, list_part_number):
    work.put_nowait((url, part_number))

def crawler():
    global df
    
    while not work.empty():
        url, part_number = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(101):
            try:
                resp = requests.get(url,
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp, 'lxml')
                
                # = = = = = = = = = = = = = = =
                
                html = etree.HTML(str(soup))
                
                if len(html.xpath('//span[contains(@class, "navlabellink") and contains(@class, "nvoffset") and contains(@class, "nnormal")]')) != 0:
                    df_temp = pd.DataFrame([{'status': 'ok',
                                             'Manufacturer': manufacturer,
                                             'Part_Number': part_number,
                                             'Url': url}])
                    
                    status = 'ok'
                    
                    break

                list_part = html.xpath('//tbody[contains(@class, "listing-inner") and (contains(@class, "altrow-a-1") or contains(@class, "altrow-a-0"))]')

                # = = = = = = = = = = = = = = =
                
                list_part_number = [_.xpath('./descendant::div[contains(@class, "listing-text-row-moreinfo-truck")]/span[contains(@title, "Buyer\'s Guide")]')[0].text for _ in list_part]
                
                # = = = = = = = = = = = = = = =
                
                number_index = np.where(np.array(list_part_number) == part_number)[0]
                
                # = = = = = = = = = = = = = = =
                
                list_manufacturer = [_.xpath('./descendant::div[contains(@class, "listing-text-row-moreinfo-truck")]/span[contains(@class, "listing-final-manufacturer")]')[0].text for _ in list_part]
                
                # = = = = = = = = = = = = = = =
                
                manufacturer_index = np.where(np.array(list_manufacturer) == manufacturer)[0]
                
                # = = = = = = = = = = = = = = =
                
                index = list(np.intersect1d(number_index, manufacturer_index))
                
                # = = = = = = = = = = = = = = =
                
                if len(index) == 1:
                    index = index[0]
                    
                    # = = = = = = = = = = = = = = =
                    
                    note_1 = list_part[index].xpath('./descendant::div[contains(@class, "listing-text-row-moreinfo-truck")]/span[contains(@class, "span-link-underline-remover")]/text()')
                    if len(note_1) == 0:
                        note_1 = ''
                    else:
                        note_1 = note_1[0].strip()
   
                    # = = = = = = = = = = = = = = =
                    
                    note_2 = list_part[index].xpath('./descendant::div[contains(@class, "listing-text-row") and not(contains(@class, "moreinfo-truck"))]/span/span/text()')
                    if len(note_2) == 0:
                        note_2 = ''
                    else:
                        note_2 = [_.strip() for _ in note_2]
                        while '' in note_2:
                            note_2.remove('')
                        note_2 = '\n'.join(note_2)

                    # = = = = = = = = = = = = = = =
                    
                    oe = list_part[index].xpath('./descendant::span[contains(@title, "Replaces these Alternate/ OE Part Numbers")]')
                    if len(oe) == 0:
                        oe = ''
                    else:
                        oe = oe[0].xpath('./text()')[0].replace(', ', ';')
                        
                    # = = = = = = = = = = = = = = =
                    
                    src = list_part[index].xpath('./descendant::img[contains(@id, "inlineimg_thumb")]')
                    if len(src) == 0:
                        src = ''
                    else:
                        src = 'https://www.rockauto.com' + src[0].xpath('./@src')[0].replace('__ra_m.', '.')
    
                    # = = = = = = = = = = = = = = =
                    
                    part_json = list_part[index].xpath('./descendant::input[contains(@id, "listing_data_essential")]')
                    if len(part_json) == 0:
                        part_key = ''
                    else:
                        part_key = json.loads(part_json[0].xpath('./@value')[0])['partkey']
                        
                    # = = = = = = = = = = = = = = =
                    
                    df_temp = pd.DataFrame([{'status': 'ok',
                                             '序号': '',
                                             'Manufacturer': manufacturer,
                                             'Part_Number': part_number,
                                             'Vehcile': '',
                                             'Alternate_OE_Part_Numbers': oe,
                                             'Note_1': note_1,
                                             'Note_2': note_2,
                                             'Pic': '',
                                             'Url': url,
                                             'Info': 'https://www.rockauto.com/en/moreinfo.php?pt=' + type_code + '&pk=' + part_key,
                                             'Src': src,
                                             'Type_Code': type_code,
                                             'Part_Key': part_key}])
                
                    status = 'ok'
                
                    break
                
            except:
                time.sleep(0.5)
                continue

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Manufacturer': manufacturer,
                                     'Part_Number': part_number,
                                     'Url': url}])

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(part_number +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(5):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df = df.drop_duplicates(ignore_index=True)

print()
print('爬虫输出中')
df.to_excel('./part_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')

总数量：448

30250SX  <->  [ok] - 剩余数量：443
715F  <->  [ok] - 剩余数量：442
30430S  <->  [ok] - 剩余数量：441
30250S  <->  [ok] - 剩余数量：440
30420S  <->  [ok] - 剩余数量：439
30460S  <->  [ok] - 剩余数量：438
30460SA  <->  [ok] - 剩余数量：437
30460SB  <->  [ok] - 剩余数量：436
30470S  <->  [ok] - 剩余数量：435
30490S  <->  [ok] - 剩余数量：434
30490SX  <->  [ok] - 剩余数量：433
30470SX  <->  [ok] - 剩余数量：432
30719SHWP  <->  [ok] - 剩余数量：431
31007S  <->  [ok] - 剩余数量：430
31035SX  <->  [ok] - 剩余数量：429
31036S  <->  [ok] - 剩余数量：428
30719S  <->  [ok] - 剩余数量：427
31036SA  <->  [ok] - 剩余数量：426
31035S  <->  [ok] - 剩余数量：425
31041S  <->  [ok] - 剩余数量：424
31045SA  <->  [ok] - 剩余数量：423
31038SX  <->  [ok] - 剩余数量：422
31045S  <->  [ok] - 剩余数量：421
31047SX  <->  [ok] - 剩余数量：420
31047SHWP  <->  [ok] - 剩余数量：419
31038S  <->  [ok] - 剩余数量：418
31047SXHWP  <->  [ok] - 剩余数量：417
31049S  <->  [ok] - 剩余数量：416
31049SA  <->  [ok] - 剩余数量：415
31047SH  <->  [ok] - 剩余数量：414
31049SAH  <->  [ok] - 剩余数量：413
31047S  <->  [ok] - 剩余数量：412
31049SBX  <->  [ok] - 剩余数量：411
31049SB  <

/var/folders/20/g09tvx4s7mdd977ps185l3_r0000gn/T/ipykernel_16611/451915924.py:96: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  number_index = np.where(np.array(list_part_number) == part_number)[0]
/var/folders/20/g09tvx4s7mdd977ps185l3_r0000gn/T/ipykernel_16611/451915924.py:104: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  manufacturer_index = np.where(np.array(list_manufacturer) == manufacturer)[0]


31058SX  <->  [ok] - 剩余数量：376
31062S  <->  [ok] - 剩余数量：375
31062SX  <->  [ok] - 剩余数量：374
31064S  <->  [ok] - 剩余数量：373
31056SX  <->  [ok] - 剩余数量：372
31065S  <->  [ok] - 剩余数量：371
31065SX  <->  [ok] - 剩余数量：370
31067S  <->  [ok] - 剩余数量：369
31069SX  <->  [ok] - 剩余数量：368
31067SX  <->  [ok] - 剩余数量：367
31073S  <->  [ok] - 剩余数量：366
31075SX  <->  [ok] - 剩余数量：365
31076SAX  <->  [ok] - 剩余数量：364
31076SX  <->  [ok] - 剩余数量：363
31076SA  <->  [ok] - 剩余数量：362
31076S  <->  [ok] - 剩余数量：361
31078S  <->  [ok] - 剩余数量：360
31090SX  <->  [ok] - 剩余数量：359
31090S  <->  [ok] - 剩余数量：358
3163S  <->  [ok] - 剩余数量：357
3160S  <->  [ok] - 剩余数量：356
3163SA  <->  [ok] - 剩余数量：355
3163SB  <->  [ok] - 剩余数量：354
31093S  <->  [ok] - 剩余数量：353
3168S  <->  [ok] - 剩余数量：352
3168SA  <->  [ok] - 剩余数量：351
31060SX  <->  [ok] - 剩余数量：350
3169S  <->  [ok] - 剩余数量：349
3168SE  <->  [ok] - 剩余数量：348
3171S  <->  [ok] - 剩余数量：347
3181SA  <->  [ok] - 剩余数量：346
3168SD  <->  [ok] - 剩余数量：345
3168SC  <->  [ok] - 剩余数量：344
3182S  <->  [ok] - 剩余数量：343
3191S  

3727SAX  <->  [ok] - 剩余数量：99
3720SA  <->  [ok] - 剩余数量：98
3727SX  <->  [ok] - 剩余数量：97
3727SB  <->  [ok] - 剩余数量：96
3727SBX  <->  [ok] - 剩余数量：95
3727S  <->  [ok] - 剩余数量：94
3730S  <->  [ok] - 剩余数量：93
3720SAX  <->  [ok] - 剩余数量：92
3730SAX1HWP  <->  [ok] - 剩余数量：91
3730SA  <->  [ok] - 剩余数量：90
3730SHWP  <->  [ok] - 剩余数量：89
3730SAX  <->  [ok] - 剩余数量：88
3730SA1HWP  <->  [ok] - 剩余数量：87
3730SAXHWP  <->  [ok] - 剩余数量：86
3730SX  <->  [ok] - 剩余数量：85
3750S  <->  [ok] - 剩余数量：84
3730SAHWP  <->  [ok] - 剩余数量：83
3752SX  <->  [ok] - 剩余数量：82
3750SX  <->  [ok] - 剩余数量：81
3752S  <->  [ok] - 剩余数量：80
3750SHVVT  <->  [ok] - 剩余数量：79
3730SXHWP  <->  [ok] - 剩余数量：78
3753S  <->  [ok] - 剩余数量：77
38MMSR130SX  <->  [ok] - 剩余数量：76
38MMSRH72S  <->  [ok] - 剩余数量：75
38MMSRH72SA  <->  [ok] - 剩余数量：74
38MMSRH62S  <->  [ok] - 剩余数量：73
38MMSRH72SB  <->  [ok] - 剩余数量：72
3753SX  <->  [ok] - 剩余数量：71
3900CS  <->  [ok] - 剩余数量：70
3900CSA  <->  [ok] - 剩余数量：69
3900CSB  <->  [ok] - 剩余数量：68
3916SA  <->  [ok] - 剩余数量：67
3900CSX  <->  [ok] - 剩余数量：66